<center>
<h1><b>Lab 2</b></h1>
<h1>PHYS 580 - Computational Physics</h1>
<h2>Professor Molnar</h2>
</br>
<h3><b>Ethan Knox</b></h3>
<h4>https://www.github.com/ethank5149</h4>
<h4>ethank5149@gmail.com</h4>
</br>
</br>
<h3><b>September 11, 2020</b></h3>
</center>

## Imports

In [13]:
import math
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from functools import partial
from scipy.integrate import solve_ivp

# Support Functions

## Euler Step

In [14]:
def euler_step(f: callable, 
               y: np.ndarray, 
               t: float, 
               dt: float, 
               *args, 
               **kwargs):
    
    return y + f(y, t, *args, **kwargs) * dt

## DSolve Function

In [15]:
def dsolve(
    fun: callable, 
    t: np.ndarray, 
    y0: np.ndarray, 
    terminate = lambda x, y : False,
    step: callable = euler_step) -> tuple:
    
    t = np.asarray(t)  # Ensure t is a Numpy array
    y = np.zeros((np.size(t), np.size(y0)))  # Create our output data container
    y[0] = y0  # Set initial condition

    dt = t[1] - t[0]  # Assume independent variable points are equidistant
    terminated_at = -1  # Index of the terminated point

    for i in range(np.size(t)-1):
        y[i+1] = euler_step(fun, y[i], t[i], dt)
        # y[i+1] = y[i] + fun(y[i], t[i]) * dt  # Step forward

        if terminate(t[i], y[i]):  # Check termination condition
            terminated_at = i  # Set termination point
            break
    
    return t[:terminated_at], y[:terminated_at, :]

# Drag Forces

## Isothermal Drag Force

In [16]:
def df_isothermal_drag(
    y: np.ndarray, 
    x: np.ndarray, 
    b2_m: float = 4.0e-5,
    y0scale: float = 1.0e4, 
    g: float = 9.81):
    
    v = np.sqrt(y[2]**2 + y[3]**2)
    drag_factor = -b2_m * v * np.exp(-y[1] / y0scale)
    
    return np.asarray([y[2],
                       y[3],
                       drag_factor * y[2], 
                       -g + drag_factor * y[3]])

## Adiabatic Drag Force

In [17]:
def df_adiabatic_drag(
    y: np.ndarray, 
    x: np.ndarray, 
    b2_m: float = 4.0e-5,
    alpha: float = 6.5e-3, 
    gamma: float = 1.4,
    T_grd: float = 293,
    g: float = 9.81):
    
    v = np.sqrt(y[2]**2 + y[3]**2)
    drag_factor = -b2_m * v * (1 - alpha * y[1]/T_grd)**(1/ (gamma-1))
    
    return np.asarray([y[2],
                       y[3],
                       drag_factor * y[2], 
                       -g + drag_factor * y[3]])

## No Drag Force

In [18]:
def df_no_drag(
    y: np.ndarray, 
    x: np.ndarray, 
    g: float = 9.81):
        
    return np.asarray([y[2],
                       y[3],
                       0, 
                       -g])

## Constant Drag Force

In [19]:
def df_constant_drag(
    y: np.ndarray, 
    x: np.ndarray, 
    b2_m: float = 4.0e-5,
    g: float = 9.81):
    
    v = np.sqrt(y[2]**2 + y[3]**2)
    drag_factor = -b2_m * v
    
    return np.asarray([y[2],
                       y[3],
                       drag_factor * y[2], 
                       -g + drag_factor * y[3]])

In [20]:
# assumes: i) drag force F = - B_2 * v^2
#          ii) isothermal air density model: rho(y) = rho(0) * exp(-y / y0)

# Shoot Trajectory Driver

## Shoot Trajectory

In [22]:
def shoot(
    v0: float = 700., 
    theta: float = 45., 
    dt: float = 0.01,
    g: float = 9.81,
    forcing: callable = df_no_drag,
    *args,
    **kwargs):


    # Convert initial conditions into an initial velocity
    vx0 = v0 * np.cos(theta / 180. * np.pi)  
    vy0 = v0 * np.sin(theta / 180. * np.pi)
    
      # Set initial condition
    y0 = np.asarray([0., 0., vx0, vy0])

    maxr = v0**2 / g   # max range of shell, in vaccuum (for automatic horizontal plot range)
    maxt = maxr / vx0  # flight time at maxr, in vacuum (for atomatic calculation end time)

    nsteps = np.round(maxt / dt)   # time steps
    t = np.linspace(0, maxt, nsteps)

    forcing = partial(forcing, *args, **kwargs)  # Fill in any specific parameters for the forcing methods
    t, soln = dsolve(forcing, t, y0, terminate=lambda _x, _y : _x > 0 and _y[1] <= 0.)
    
    # Post Analysis
    x, y, dx, dy = soln.T
    
    max_range = (y[-1] * x[-2] - y[-2] * x[-1] ) / (y[-1] - y[-2])
    x[-1] = max_range
    y[-1] = 0    

    max_height_index = np.argmax(y)
    max_height = y[max_height_index]
    x_at_max_height = x[max_height_index]
    
    return t, x, y, dx, dy, x_at_max_height, max_height, max_range

# Simulation

In [25]:
angles = np.linspace(5, 85, 1000)

no_drag_ranges = np.asarray([shoot(angle, forcing=df_no_drag) for angle in angles])
constant_drag_ranges = np.asarray([shoot(angle, forcing=df_constant_drag) for angle in angles])
adiabatic_drag_ranges = np.asarray([shoot(angle, forcing=df_adiabatic_drag) for angle in angles])
isothermal_drag_ranges = np.asarray([shoot(angle, forcing=df_isothermal_drag) for angle in angles])

no_drag_optimum_angle_index = np.argmax(no_drag_ranges)
constant_drag_optimum_angle_index = np.argmax(constant_drag_ranges)
adiabatic_drag_optimum_angle_index = np.argmax(adiabatic_drag_ranges)
isothermal_drag_optimum_angle_index = np.argmax(isothermal_drag_ranges)

no_drag_optimum_angle = angles[no_drag_optimum_angle_index]
constant_drag_optimum_angle = angles[constant_drag_optimum_angle_index]
adiabatic_drag_optimum_angle = angles[adiabatic_drag_optimum_angle_index]
isothermal_drag_optimum_angle = angles[isothermal_drag_optimum_angle_index]

no_drag_optimum = shoot(forcing=df_no_drag, theta=no_drag_optimum_angle)
constant_drag_optimum = shoot(forcing=df_constant_drag, theta=constant_drag_optimum_angle)
adiabatic_drag_optimum = shoot(forcing=df_adiabatic_drag, theta=adiabatic_drag_optimum_angle)
isothermal_drag_optimum = shoot(forcing=df_isothermal_drag, theta=isothermal_drag_optimum_angle)

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

# Plot

## Range vs. Launch Angle

In [ ]:
plt.xlabel('Angle [degrees]')
plt.ylabel('Range [m]')
plt.suptitle("Ranges")
plt.plot(angles, no_drag_ranges, label = "No Drag")
plt.plot(angles, constant_drag_ranges, label = "Constant Drag")
plt.plot(angles, adiabatic_drag_ranges, label = "Adiabatic Drag")
plt.plot(angles, isothermal_drag_ranges, label = "Isothermal Drag")
plt.legend()
plt.grid()
plt.show()

## Trajectory At Optimum Launch Angle

In [ ]:
#theta=44.95995995995996
#theta=38.793793793793796
#theta=43.67867867867868
#theta=45.92092092092092
fig = plt.figure(figsize=(16,9), constrained_layout=True)
plt.xlabel('x [m]')
plt.ylabel('y [m]')
plt.suptitle("Trajectories")
plt.plot(no_drag_optimum.x, no_drag_optimum.y, label = "No Drag")
plt.plot(constant_drag_optimum.x, constant_drag_optimum.y, label = "Constant Drag")
plt.plot(adiabatic_drag_optimum.x, adiabatic_drag_optimum.y, label = "Adiabatic Drag")
plt.plot(isothermal_drag_optimum.x, isothermal_drag_optimum.y, label = "Isothermal Drag")
plt.legend()
plt.grid()
plt.show()